In [55]:
from Bio import SeqIO
import numpy as np
import allel
import matplotlib.pyplot as plt
import pandas as pd

In [41]:
import os
os.getcwd()

'/home/alexcc/notebooks'

In [44]:
### Read in GFF
f = open('../covid19_population_genomics/reference_genome/reference.gff')
genes = {}
for line in f.readlines():
    if not line.startswith("#"):
        if line.split("\t")[2] == 'CDS':
            gene = line.split(";product=")[1].split(";")[0]
            start = int(line.split("\t")[3])
            stop = int(line.split("\t")[4])
            genes[gene] = {"start": start, "stop": stop}

### Create alignment to GFF index
ref = 'MN908947'
seqs = []

for record in SeqIO.parse('../covid19_population_genomics/interpatient/mar20_filtered.aln', 'fasta'):
    if record.id == ref:
        ref_record = record
    seqs.append(str(record.seq).upper())

align_index = {}
true_pos = 1
align_pos = 1
for pos in ref_record:
    if pos != '-':
        align_index[align_pos] = true_pos
        true_pos += 1
    else:
        align_index[align_pos] = np.nan
    align_pos += 1

In [45]:
## Create scikit-alllel object

P2C = {'A':0, 'C':1, 'T':2, 'G':3}
C2P = {0:'A', 1:'C', 2:'T', 3:'G'}

allele_counts = []
align_pos = 1
for pos in ref_record:
    alleles = [0,0,0,0]
    if not np.isnan(align_index[align_pos]):
        for seq in seqs:
            if seq[align_pos-1] in P2C:
                alleles[P2C[seq[align_pos-1]]] += 1


        allele_counts.append(alleles)
    align_pos += 1
allel1 = allel.AlleleCountsArray(allele_counts)

In [46]:
positions = [x for x in range(1, len(allel1)+1)]
nuc_div = allel.windowed_diversity(positions, allel1, start=1, size=1, step=1)

In [56]:
## calculate pi for each protein product
protein_products = pd.read_table('../covid19_population_genomics/reference_genome/COVID_genes_table_v3.tsv', sep="\t")
gene_names = []
genes_pi = []

for index, p in protein_products.iterrows():
    nuc_div = allel.sequence_diversity(range(p['start'], p['end']+1), allel1)
    gene_names.append(p['name'])
    genes_pi.append(nuc_div)

In [60]:
sites = 0 
for p in positions:
    if np.sum(allel1[p-1].astype(bool)) > 1:
        for nuc in range(0,4):
            if allel1[p-1][nuc] > 0:
                print(str(p) + "\t" + str(C2P[nuc]) + "\t" + str(allel1[p-1][nuc] / np.sum(allel1[p-1])))
        sites += 1

1	A	0.9333333333333333
1	C	0.05333333333333334
1	T	0.013333333333333334
2	C	0.01282051282051282
2	T	0.9871794871794872
3	C	0.012658227848101266
3	T	0.9746835443037974
3	G	0.012658227848101266
4	A	0.9753086419753086
4	T	0.012345679012345678
4	G	0.012345679012345678
5	A	0.9876543209876543
5	G	0.012345679012345678
6	A	0.9876543209876543
6	T	0.012345679012345678
7	A	0.024390243902439025
7	T	0.012195121951219513
7	G	0.9634146341463414
12	A	0.9880952380952381
12	T	0.011904761904761904
15	A	0.011235955056179775
15	C	0.9887640449438202
16	C	0.978021978021978
16	T	0.02197802197802198
17	C	0.01098901098901099
17	T	0.989010989010989
28	C	0.9894736842105263
28	T	0.010526315789473684
35	A	0.9896907216494846
35	T	0.010309278350515464
36	C	0.9896907216494846
36	T	0.010309278350515464
75	A	0.009708737864077669
75	C	0.9902912621359223
104	A	0.009708737864077669
104	T	0.9902912621359223
111	C	0.009708737864077669
111	T	0.9902912621359223
112	T	0.9902912621359223
112	G	0.009708737864077669
119	C	0.990291

In [61]:
sites

200